In [2]:
import psycopg2
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import csv

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


# Parte 1

In [3]:
conn = psycopg2.connect("dbname=javier_rojas user=postgres password=2719")
cursor = conn.cursor()

In [4]:
query_train = """
CREATE TABLE train_cupid(
 age INTEGER,
 height DECIMAL,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs DECIMAL,
 pro_cats DECIMAL,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

query_test = """
CREATE TABLE test_cupid(
 age INTEGER,
 height DECIMAL,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs DECIMAL,
 pro_cats DECIMAL,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

cursor.execute(query_train)
cursor.execute(query_test)

conn.commit()

In [5]:
with open("input/train_cupid.csv") as file:
    reader = csv.reader(file)
    next(reader)

    for row in reader:
        cursor.execute(
            "INSERT INTO train_cupid VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row,
        )


In [6]:
with open("input/test_cupid.csv") as file:
    reader = csv.reader(file)
    next(reader)

    for row in reader:
        cursor.execute(
            "INSERT INTO test_cupid VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row,
        )

In [7]:
conn.commit()

# Parte 2

In [16]:
COLUMNAS = [
    "age",
    "height",
    "virgo",
    "taurus",
    "scorpio",
    "pisces",
    "libra",
    "leo",
    "gemini",
    "aries",
    "aquarius",
    "cancer",
    "sagittarius",
    "asian",
    "hispaniclatin",
    "black",
    "indian",
    "pacificislander",
    "nativeamerican",
    "middleeastern",
    "colorado",
    "newyork",
    "oregon",
    "arizona",
    "hawaii",
    "montana",
    "wisconsin",
    "virginia",
    "spain",
    "nevada",
    "illinois",
    "vietnam",
    "ireland",
    "louisiana",
    "michigan",
    "texas",
    "unitedkingdom",
    "massachusetts",
    "northcarolina",
    "idaho",
    "mississippi",
    "newjersey",
    "florida",
    "minnesota",
    "georgia",
    "utah",
    "washington",
    "westvirginia",
    "connecticut",
    "tennessee",
    "rhodeisland",
    "districtofcolumbia",
    "canada",
    "missouri",
    "germany",
    "pennsylvania",
    "netherlands",
    "switzerland",
    "mexico",
    "ohio",
    "agnosticism",
    "atheism",
    "catholicism",
    "buddhism",
    "judaism",
    "hinduism",
    "islam",
    "pro_dogs",
    "pro_cats",
    "spanish",
    "chinese",
    "french",
    "german",
    "single",
    "seeing_someone",
    "available",
    "employed",
    "income_between_25_50",
    "income_between_50_75",
    "income_over_75",
    "drugs_often",
    "drugs_sometimes",
    "drinks_notatall",
    "drinks_often",
    "drinks_rarely",
    "drinks_socially",
    "drinks_veryoften",
    "orientation_gay",
    "orientation_straight",
    "sex_m",
    "smokes_sometimes",
    "smokes_tryingtoquit",
    "smokes_whendrinking",
    "smokes_yes",
    "body_type_overweight",
    "body_type_regular",
    "education_high_school",
    "education_undergrad_university",
]


In [17]:
cursor.execute("SELECT * FROM train_cupid;")
train = pd.DataFrame(cursor.fetchall(), columns=COLUMNAS)

In [19]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB